# Requirements

### installations

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 5.1 MB/s 
     |████████████████████████████████| 4.9 MB 59.5 MB/s 
     |████████████████████████████████| 212 kB 74.7 MB/s 
     |████████████████████████████████| 115 kB 51.9 MB/s 
     |████████████████████████████████| 163 kB 71.6 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [3]:
! pip install optuna
! pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 5.0 MB/s 
     |████████████████████████████████| 209 kB 66.4 MB/s 
     |████████████████████████████████| 81 kB 11.5 MB/s 
     |████████████████████████████████| 78 kB 8.2 MB/s 
     |████████████████████████████████| 112 kB 64.4 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 147 kB 63.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=137402255db72094d5db8f0e85c0b00f3ed6b707cb76585e40f100fc0ef5d57b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 2.2 MB/s 
     |████████████████████████████████| 4.1 MB 42.2 

### dependencies

In [4]:
import transformers

print(transformers.__version__, transformers.__version__ >= '4.11.0')

4.22.2 True


In [5]:
import datasets
import random
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [6]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


# Define parms

In [7]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [8]:
task = "mnli" #"cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

In [9]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

In [25]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

# Get data

In [10]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
dataset 

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [15]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(10000))

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 10000
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [19]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [20]:
show_random_elements(dataset["train"])

,premise,hypothesis,label,idx
0,"In the morning, when his tired and still crying wife fell asleep in the locked bathroom, the exhausted professor sat down on the sofa, and said to himself:",The professor sat down on the sofa at night.,contradiction,98721
1,"Sulfates are a key factor in all areas of the United States, particularly in the East, where high humidity increases the light extinction efficiency of sulfates.",High humidity causes many problems in the Eastern United States.,neutral,322782
2,"Ca'daan's eyes fell to the small man, Stark.",Ca'daan looked at the small man.,entailment,362440
3,"And as I've said before, and shall doubtless say again, little Tuppence can look after herself, thank you!"" And with a short, sharp nod of her head she walked briskly onward.",Tuppence tripped and fell after walking away.,neutral,24089
4,You should try the Singapore specials and you must not miss dessert.,We no longer offer the Singapore specials.,contradiction,143712
5,"Today, pseudo-scientific theories about the origin and purpose of the pyramids fill the stands of bookshops and the listings on documentary channels.","Nobody cares about the pyramids, and theories about them and their purpose usually fade into obscurity.",contradiction,287322
6,"Although the main building is not open to the public, the official residence of the Portuguese government may be visited by prearranged, guided tours.",It's not unusual for tour guides to sneak into the Portugese government's official residence.,neutral,239723
7,you know i mean one of the things that we talked about that i truly believe is you know you give somebody a you know a jury convicts somebody and they give them a sixty year sentence and the guys going to be out in twelve or thirteen years you know,All convicts are forced to spend the whole duration of their sentencing in prison.,contradiction,312792
8,"Creating a lush Madeiran backyard won't be easy, but you can still take home souvenir flamingo flowers (anthuriums), orchids, and bird-of-paradise flowers (strelitzias, or estrel?­cias in Portuguese).",The bird-of-paradise is a kind of flower that is also known as strelitzias.,entailment,300347
9,"'Dorcas,' she says, 'I've had a great shock.' 'I'm sorry for that, m'm,' I says.",Dorcas is sorry that the speaker has had a great shock,entailment,239100


In [21]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [22]:
# SAMPLE ONLY
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5}

# Preprocessing

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [24]:
# SAMPLE
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [26]:
sentence1_key, sentence2_key = task_to_keys[task]
print(task)
print(sentence1_key)
print(sentence2_key)

mnli
premise
hypothesis


In [27]:
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: I'll hurry over that part.
Sentence 2: "I'll be quick with that part."


In [28]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 10000
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [30]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [31]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9847
    })
})

# Fine tuning

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [33]:
model_name = model_checkpoint.split("/")[-1]

In [50]:
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5, #10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [40]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
print(validation_key)
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

validation_matched


/content/distilbert-base-uncased-finetuned-mnli is already a clone of https://huggingface.co/charlemagne/distilbert-base-uncased-finetuned-mnli. Make sure you pull the latest changes with `repo.git_pull()`.


In [41]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6250


Epoch,Training Loss,Validation Loss,Accuracy
1,0.312800,1.121862,0.677229
2,0.180100,1.436558,0.671625
3,0.130400,1.846045,0.677738
4,0.137700,1.969065,0.670199
5,0.094900,2.135082,0.679165
6,0.069300,2.371800,0.679674
7,0.058100,2.462861,0.680082
8,0.033300,2.489574,0.681202
9,0.023900,2.579315,0.678349
10,0.014800,2.596244,0.679063


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-mnli/checkpoint-625
Configuration saved in distilbert-base-uncased-finetuned-mnli/checkpoint-625/config.json
Model weights saved in distilbert-base-uncased-finetuned-mnli/checkpoint-625/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/checkpoint-625/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-mnli/checkpoint-625/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/tokenizer_config.json
Special tokens file saved in distilbert-base-unc

TrainOutput(global_step=6250, training_loss=0.09735435958862304, metrics={'train_runtime': 1108.8311, 'train_samples_per_second': 90.185, 'train_steps_per_second': 5.637, 'total_flos': 2097633827734848.0, 'train_loss': 0.09735435958862304, 'epoch': 10.0})

In [42]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16


{'eval_loss': 2.4895739555358887,
 'eval_accuracy': 0.6812022414671421,
 'eval_runtime': 24.8632,
 'eval_samples_per_second': 394.759,
 'eval_steps_per_second': 24.695,
 'epoch': 10.0}

# Pushing fine-tuned model to Hub (for easy access next time)

In [43]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-mnli
Configuration saved in distilbert-base-uncased-finetuned-mnli/config.json
Model weights saved in distilbert-base-uncased-finetuned-mnli/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-mnli/special_tokens_map.json
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/255M [00:00<?, ?B/s]

Upload file runs/Sep30_10-00-33_0649c393d755/events.out.tfevents.1664532044.0649c393d755.69.2:  37%|###6      …

Upload file runs/Sep30_10-00-33_0649c393d755/events.out.tfevents.1664533263.0649c393d755.69.4: 100%|##########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/charlemagne/distilbert-base-uncased-finetuned-mnli
   44b5827..3ebf93f  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/charlemagne/distilbert-base-uncased-finetuned-mnli
   44b5827..3ebf93f  main -> main

To https://huggingface.co/charlemagne/distilbert-base-uncased-finetuned-mnli
   3ebf93f..efff7d0  main -> main

   3ebf93f..efff7d0  main -> main



'https://huggingface.co/charlemagne/distilbert-base-uncased-finetuned-mnli/commit/3ebf93f943f0c62e602d63b3e521d526ff6eb9f1'

In [44]:
from transformers import AutoModelForSequenceClassification

my_awesome_model = AutoModelForSequenceClassification.from_pretrained("charlemagne/distilbert-base-uncased-finetuned-mnli")

Downloading:   0%|          | 0.00/769 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--charlemagne--distilbert-base-uncased-finetuned-mnli/snapshots/efff7d02f3f68952d8e8eaf87f320f7a10f4393e/config.json
Model config DistilBertConfig {
  "_name_or_path": "charlemagne/distilbert-base-uncased-finetuned-mnli",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--charlemagne--distilbert-base-uncased-finetuned-mnli/snapshots/efff7d02f3f68952d8e8eaf87f320f7a10f4393e/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at charlemagne/distilbert-base-uncased-finetuned-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


# Hyperparameter search

In [47]:
model_checkpoint

'distilbert-base-uncased'

In [45]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [51]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"].shard(index=1, num_shards=10),
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.2",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/

In [52]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2022-09-30 10:35:11,523] A new study created in memory with name: no-name-44f66c72-b6a2-4e74-ae0f-b3cad1bc4fb6
Trial:
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.2",
  "vocab_size": 30

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.101914,0.327866
2,No log,1.100756,0.330616
3,No log,1.100491,0.332247


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-mnli/run-0/checkpoint-32
Configuration saved in distilbert-base-uncased-finetuned-mnli/run-0/checkpoint-32/config.json
Model weights saved in distilbert-base-uncased-finetuned-mnli/run-0/checkpoint-32/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/run-0/checkpoint-32/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-mnli/run-0/checkpoint-32/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/tokenizer_config.json
Special tokens file sav

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.101486,0.326439


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-mnli/run-1/checkpoint-32
Configuration saved in distilbert-base-uncased-finetuned-mnli/run-1/checkpoint-32/config.json
Model weights saved in distilbert-base-uncased-finetuned-mnli/run-1/checkpoint-32/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/run-1/checkpoint-32/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-mnli/run-1/checkpoint-32/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from disti

...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
[I 2022-09-30 10:51:22,677] Trial 7 pruned. 
Trial:
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABE

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.096434,0.360774


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
[I 2022-09-30 10:51:56,609] Trial 8 pruned. 
Trial:
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABE

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.098654,0.336220
2,No log,1.097984,0.342537


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-mnli/run-9/checkpoint-32
Configuration saved in distilbert-base-uncased-finetuned-mnli/run-9/checkpoint-32/config.json
Model weights saved in distilbert-base-uncased-finetuned-mnli/run-9/checkpoint-32/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/run-9/checkpoint-32/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-mnli/run-9/checkpoint-32/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassifica

In [53]:
best_run

BestRun(run_id='2', objective=0.5485481406011208, hyperparameters={'learning_rate': 6.561831502631091e-06, 'num_train_epochs': 5, 'seed': 16, 'per_device_train_batch_size': 4})

In [54]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.2",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.092560,0.355884
2,1.075600,1.025713,0.502598
3,1.075600,0.981288,0.532552
4,0.883600,0.956142,0.545695
5,0.883600,0.952584,0.548548


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-mnli/checkpoint-250
Configuration saved in distilbert-base-uncased-finetuned-mnli/checkpoint-250/config.json
Model weights saved in distilbert-base-uncased-finetuned-mnli/checkpoint-250/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-mnli/checkpoint-250/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-mnli/checkpoint-250/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have be

TrainOutput(global_step=1250, training_loss=0.9370080932617187, metrics={'train_runtime': 205.5053, 'train_samples_per_second': 24.33, 'train_steps_per_second': 6.083, 'total_flos': 78069190803480.0, 'train_loss': 0.9370080932617187, 'epoch': 5.0})